<a href="https://colab.research.google.com/github/edcalderin/DeepLearning_SaturdaysAI/blob/master/Tareas/Tarea_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea: GAN
### Grupo: XXXXX
Integrantes:
<br>
* Integrante 1 
* Integrante 2
* Integrante 3
* Integrante 4
<br>

Indicaciones:
<rb>
* Debe realizar la siguiente tarea hasta el 04 de julio, 23:59 UTC - 4
* Debe hacer una copia de este notebook para poder editar el código.
* Una vez finalizado el trabajo debe subir el link de su notebook (con permisos de lector) en la sección de "Tareas" del módulo correspondiente

Para la tarea se puede elegir una de tres opciones:


A) Partiendo de la red GAN vista en clase tomar otro dataset y entrenar para él. Por ejemplo se puede usar el dataset: https://ai.stanford.edu/~jkrause/cars/car_dataset.html

B) Partiendo de la red GAN vista en clase, utilizando MNIST, trabajar con la red DCGAN: https://arxiv.org/pdf/1511.06434.pdf

C) Realizar A y B al mismo tiempo


Se deja a modo de ejemplo parte del código de un posible discriminador para la opción B) a continuación y código auxiliar para obtener el dataset de A).

Se recomienda si se realizan entrenamientos muy largos ejecutar el código en una instancia cloud, no en Colab.


In [ ]:
from torchvision.datasets.utils import download_url
import tarfile
dataset_url = 'https://s3.amazonaws.com/fast-ai-imageclas/stanford-cars.tgz'
download_url(dataset_url, '.')
with tarfile.open('./stanford-cars.tgz', 'r:gz' )as tar:
                  tar.extractall(path= './cars_data')
data_dir = './cars_data/stanford-cars'
train_dir = data_dir +'/cars_train'
print(os.listdir(data_dir))
len(train_dir)

In [ ]:
IMAGE_DIM = (32, 32, 3)
def get_sample_image(G, n_noise):
    """
        save sample 100 images
    """
    z = torch.randn(10, n_noise).to(DEVICE)
    y_hat = G(z).view(10, 3, 32, 32).permute(0, 2, 3, 1) # (100, 28, 28)
    result = (y_hat.detach().cpu().numpy()+1)/2.
    return result

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, num_classes=1):
        super(Discriminator, self).__init__()
        self.conv = nn.Sequential(
            # Reduce size
            nn.Conv2d(in_channel, 512, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            # Reduce size
            # @TODO

            # Reduce size
            # @TODO
            
            # 
            nn.Conv2d(128, 128, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1),
        )        
        self.fc = nn.Sequential(
            # reshape input, 128 -> 1
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x, y=None):
        y_ = self.conv(x)
        y_ = y_.view(y_.size(0), -1)
        y_ = self.fc(y_)
        return y_

In [ ]:
# Recordar que a diferencia de MNIST ahora se está trabajando con imágenes a color por lo que cambian las dimensiones
transform = transforms.Compose([transforms.Resize((IMAGE_DIM[0],IMAGE_DIM[1])),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))
                               ]
